In [1]:
import math, pylab, csv
import xarray as xr
import numpy as np
from datetime import datetime
from datetime import date
from itertools import groupby
from collections import Counter
from matplotlib.path import Path
import matplotlib.pyplot as plt
import earthaccess
import xarray as xr
from xarray.backends.api import open_datatree
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
#%matplotlib widget
from scipy.ndimage import generic_filter
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata
import seaborn as sns 
import pandas
from xarray.backends.api import open_datatree
import numpy as np

In [2]:
# %matplotlib widget

In [3]:
fontsize = 20

plt.rc('font', size=fontsize)          # controls default text sizes
plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('legend', fontsize=fontsize)    # legend fontsize
plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title

In [4]:
ds = xr.open_dataset('Edward_Eddy_trajectory_nrt_3.2exp_cyclonic_20180101_20240723.nc')
ds

<xarray.Dataset> Size: 202kB
Dimensions:                        (obs: 223, NbSample: 20)
Dimensions without coordinates: obs, NbSample
Data variables: (12/27)
    amplitude                      (obs) float64 2kB ...
    effective_area                 (obs) float32 892B ...
    effective_contour_height       (obs) float32 892B ...
    effective_contour_latitude     (obs, NbSample) float64 36kB ...
    effective_contour_longitude    (obs, NbSample) float64 36kB ...
    effective_contour_shape_error  (obs) float64 2kB ...
    ...                             ...
    speed_contour_longitude        (obs, NbSample) float64 36kB ...
    speed_contour_shape_error      (obs) float64 2kB ...
    speed_radius                   (obs) float64 2kB ...
    time                           (obs) datetime64[ns] 2kB ...
    track                          (obs) uint32 892B ...
    uavg_profile                   (obs, NbSample) float64 36kB ...
Attributes: (12/19)
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    comment:                   Surface product; mesoscale eddies
    creator_email:             aviso@altimetry.fr
    creator_url:               https://www.aviso.altimetry.fr
    date_created:              2024-08-06T09:38:06Z
    framework_used:            https://github.com/AntSimi/py-eddy-tracker
    ...                        ...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   This dataset contains eddy atlas from all-sate...
    time_coverage_duration:    P2396D
    time_coverage_end:         2024-07-23T00:00:00Z
    time_coverage_start:       2018-01-01T00:00:00Z
    title:                     Mesoscale Cyclonic Eddies in Altimeter Observa...

In [5]:
def in_eddy(ds,float_lat,float_lon,float_time):
    """
    float_lat: degrees north
    float_lon: degrees east
    float_time: should be in format 'YYYY-MM-DD'
    """

    def all_equal(iterable):
        g = groupby(iterable)
        return next(g, True) and not next(g, False)

    in_eddy_flag = False 

    float_time = np.datetime64(float_time)
    if float_time in ds.time: # some dates not in dateset        
        for i in np.where(ds.time == float_time)[0]: # usually will only be 1 eddy, but sometimes there are 2 after a split
            contour_lons = np.array(ds.effective_contour_longitude[i]) # eddy lons
            contour_lats = np.array(ds.effective_contour_latitude[i]) # eddy lats
            
            if all_equal(contour_lons): # eddy break
                pass
            else:
                poly = Path([(contour_lats[j],contour_lons[j]) for j in np.arange(0,len(contour_lats))]) # set up the polygon
                if poly.contains_points([(float_lat,float_lon)]): #find if point is inside the polygon
                    in_eddy_flag = True 

    return in_eddy_flag

In [6]:
# in_eddy(ds,35,289.5,'2024-05-08')

In [7]:
satdate,N,chl_mean,chl_std,poc_mean,poc_std=[],[],[],[],[],[];

In [8]:
# PACE CHL
results = earthaccess.search_datasets(instrument="oci")

tspan = ("2024-04-19", "2024-10-18")
bbox = (-73, 34, -70, 37.5) #edward
clouds = (0, 100)
results = earthaccess.search_data(
    short_name="PACE_OCI_L2_BGC_NRT",
    temporal=tspan,
    bounding_box=bbox,
    cloud_cover=clouds,
)

paths = earthaccess.open(results) #'streaming' data

QUEUEING TASKS | :   0%|          | 0/273 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/273 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/273 [00:00<?, ?it/s]

In [9]:
len(paths)

273

In [ ]:
for IDX in range(261,len(paths)): #FOR EACH IMAGE. had to do in chunks 
#for IDX in range(0,1): #FOR EACH IMAGE
    datatree = open_datatree(paths[IDX])
    dataset = xr.merge(datatree.to_dict().values())
    dataset = dataset.set_coords(("longitude", "latitude"))
    h=str(paths[IDX])
    sat_date=h[64:72]
    mask = (dataset.longitude >= bbox[0]) & (dataset.longitude <= bbox[2]) & (dataset.latitude >= bbox[1]) & (dataset.latitude <= bbox[3])
    if mask.sum() == 0:
        mask = (dataset.longitude >= bbox[0]) & (dataset.longitude <= bbox[2])
        if mask.sum() == 0:
            print('Mask via longitude only')
            mask = (dataset.latitude >= bbox[1]) & (dataset.latitude <= bbox[3])
            print(mask.sum())
        else:
            print('Mask via latitude only')
            print(mask.sum())
    ds_masked = dataset.where(mask, drop=True)
    lat=ds_masked.latitude.values.flatten()
    lon=ds_masked.longitude.values.flatten()+360
    chl=ds_masked.chlor_a.values.flatten()
    poc=ds_masked.poc.values.flatten()
    
    chl_inside,poc_inside=[],[]
    
    for index in np.arange(0,len(lat)):
        val=in_eddy(ds,lat[index],lon[index],'2024-05-08')
        if val:
            chl_inside.append(chl[index]) 
            poc_inside.append(poc[index])
    
    N1=np.size(np.where(np.isnan(chl_inside)==False))

    if N1>5000:
        chlmean=np.nanmean(chl_inside);
        chlstd=np.nanstd(chl_inside);
        pocmean=np.nanmean(poc_inside);
        pocstd=np.nanstd(poc_inside);
        
        chl_mean.append(chlmean);
        poc_mean.append(pocmean);
        chl_std.append(chlstd);
        poc_std.append(pocstd);
        N.append(N1);
        satdate.append(sat_date);

Mask via latitude only
<xarray.DataArray 'longitude' ()> Size: 8B
array(230957)


In [ ]:
d = {'satdate': satdate, 'N':N, 'chlmean': chl_mean, 'pocmean': poc_mean, 'chlstd': chl_std, 'pocstd': poc_std}
data = pandas.DataFrame(data=d)
data

In [ ]:
data.to_csv('CHL_TIMESERIES_PACE.csv')

In [ ]:
IDX